In [1]:
import os
for env in ["MKL_NUM_THREADS", "NUMEXPR_NUM_THREADS", "OMP_NUM_THREADS"]:
    os.environ[env] = "1"

import numpy as np
from numpy.random import default_rng
import numpy.random as rnd
from ttictoc import tic, toc

import approxbayescomp as abc

In [2]:
import sys

print("ABC version:", abc.__version__)
print("Python version:", sys.version)
print("Numpy version:", np.__version__)

tic()

ABC version: 0.1.0
Python version: 3.8.8 | packaged by conda-forge | (default, Feb 20 2021, 15:50:57) 
[Clang 11.0.1 ]
Numpy version: 1.20.2


In [3]:
numIters = numItersData = 10
popSize = 1000
smcArgs = {"verbose": True, "numProcs": 8}

## Inference of a Negative Binomial - Weibull model

In this notebook we are are conducting a simulation experiment where the claim frequency are Negative Binomial distributed 

$$
n_s\underset{\textbf{i.i.d.}}{\sim}\text{Neg-Bin}(\alpha = 4, p = 2/3),\text{ }s = 1,\ldots, 30
$$ 

and the individual claim sizes are weibull distributed

$$
u_1,\ldots, u_{n_s}\underset{\textbf{i.i.d.}}{\sim}\text{Weib}(k = 1/2, \beta = 1),\text{ }s = 1,\ldots 30.
$$ 

The available data is aggregated claim sizes in excess of the priority $c=1$ asociated to aa global stop-loss treaty, we have 

$$
x_s = \left(\sum_{k = 1}^{n_s}u_k-c\right)_{+},\text{ }s = 1,\ldots, t.
$$

Our aim is to look into the finite sample performance of our ABC implementation when the model is well specified that is when we assume a negative binomial - weibull model

In [4]:
rg = default_rng(123)

sample_sizes = [50, 250]
T = sample_sizes[-1]
t = np.arange(1, T + 1, 1)

# Frequency-Loss Model
α, p, k, β = 4, 2 / 3, 1 / 3, 1
θ_True = α, p, k, β
θ_sev = k, β
θ_freq = α, p
freq = "negative binomial"
sev = "weibull"

# Aggregation process
c = 0.25
psi = abc.Psi("GSL", c)

freqs, sevs = abc.simulate_claim_data(rg, T, freq, sev, θ_True)
xData = abc.compute_psi(freqs, sevs, psi)

In [5]:
[np.sum(xData[:ss] > 0) for ss in sample_sizes]

[30, 155]

In [6]:
params = ("α", "p", "r", "m")
prior = abc.IndependentUniformPrior([(0, 10), (1e-3, 1), (0, 10), (0, 50)], params)
model = abc.Model("negative binomial", "gamma", psi, prior)

In [7]:
for ss in sample_sizes:
    xDataSS = xData[:ss]
    %time fit = abc.smc(numIters, popSize, xDataSS, model, **smcArgs)

Final population dists <= 9.32, ESS = 649
CPU times: user 3.9 s, sys: 317 ms, total: 4.21 s
Wall time: 58.3 s


Final population dists <= 6.02, ESS = 776
CPU times: user 4.01 s, sys: 356 ms, total: 4.37 s
Wall time: 2min 8s


In [8]:
params = ("r", "m")
prior = abc.IndependentUniformPrior([(0, 10), (0, 50)], params)

In [9]:
for ss in sample_sizes:
    xDataSS = xData[:ss]
    nData = freqs[:ss]

    model = abc.Model(nData, "gamma", psi, prior)

    %time fit = abc.smc(numItersData, popSize, xDataSS, model, **smcArgs)

Final population dists <= 7.13, ESS = 893
CPU times: user 2.5 s, sys: 166 ms, total: 2.67 s
Wall time: 1min 49s


Final population dists <= 4.97, ESS = 878
CPU times: user 9.7 s, sys: 904 ms, total: 10.6 s
Wall time: 5min 50s


In [10]:
elapsed = toc()
print(f"Notebook time = {elapsed:.0f} secs = {elapsed/60:.2f} mins")

Notebook time = 648 secs = 10.80 mins
